# CardPay

In [46]:
sequenceDiagram
    actor User as Client
    participant Shop
    participant CardPay
    participant Bank as Client's bank

    User->>Shop: Choose products
    User->>Shop: Complete order
    activate Shop
    Shop->>Shop: Generate URL parameters
    Shop->>User: Redirect to payment gate
    User->>CardPay: 
    User->>CardPay: Enter credit card details
    CardPay->>Bank: Process transaction
    Bank->>CardPay: 
    CardPay->>User: Redirect to shop
    User->>Shop: 
    Shop->>Shop: Verify URL parameters
    Shop->>User: Order is paid
    deactivate Shop

## Setup

In [11]:
// nugets, imports
#r "nuget:BouncyCastle.Cryptography"

Installed Packages BouncyCastle.Cryptography, 2.2.1

In [12]:
// usings, configuration

using System.Globalization;
using System.IO;
using System.Text.Json;
using System.Text.Json.Nodes;
using Org.BouncyCastle.Crypto;
using Org.BouncyCastle.Crypto.Parameters;
using Org.BouncyCastle.OpenSsl;
using Org.BouncyCastle.Security;

#!value --name keys_json --from-file ./keys.json

In [13]:
// Utils - formatting and crypto
public class Utils
{
    public static byte[] ConvertHexStringToByteArray(string hexString)
    {
        if (hexString.Length % 2 != 0)
        {
            throw new ArgumentException(String.Format(CultureInfo.InvariantCulture, "The binary key cannot have an odd number of digits: {0}", hexString));
        }

        byte[] data = new byte[hexString.Length / 2];
        for (int index = 0; index < data.Length; index++)
        {
            string byteValue = hexString.Substring(index * 2, 2);
            data[index] = byte.Parse(byteValue, NumberStyles.HexNumber, CultureInfo.InvariantCulture);
        }

        return data; 
    }

    public static string ConvertByteArrayToHexString(byte[] buffer)
    {
        string s = BitConverter.ToString(buffer).Replace("-", "").ToLowerInvariant();
        return s;
    }

    public static byte[] CalculateHMAC(string message, byte[] key)
    {
        var fn = new System.Security.Cryptography.HMACSHA256(key);
        byte[] HMAC_output = fn.ComputeHash(Encoding.ASCII.GetBytes(message));
        return HMAC_output;
    }

    public static byte[] CalculateSignature(string message, string PRIVATE_PEM)
    {
        var parameters = new PemReader(new StringReader(PRIVATE_PEM)).ReadObject();
        var signer = SignerUtilities.GetSigner("SHA256withECDSA");
        // signer.Init(true, ((AsymmetricCipherKeyPair)bankPrivateKey).Private);
        signer.Init(true, (ECPrivateKeyParameters)parameters);
        byte[] responseToVerify_Bytes = Encoding.UTF8.GetBytes(message);
        signer.BlockUpdate(responseToVerify_Bytes, 0, responseToVerify_Bytes.Length);
        byte[] signature = signer.GenerateSignature();
        return signature;
        // string signature = Utils.ConvertByteArrayToHexString(signedResponse);
        // signature.Display();
    }

    public static bool IsSignatureValid(string message, byte[] signature, string PUBLIC_PEM)
    {
        var paramters = new PemReader(new StringReader(PUBLIC_PEM)).ReadObject();
        var signer = SignerUtilities.GetSigner("SHA256withECDSA");
        signer.Init(false, (ICipherParameters)paramters);
        byte[] responseToVerify_Bytes = Encoding.UTF8.GetBytes(message);
        signer.BlockUpdate(responseToVerify_Bytes, 0, responseToVerify_Bytes.Length);
        bool isSignatureValid = signer.VerifySignature(signature);
        return isSignatureValid;
    }
}

In [14]:
// configuration
string MID = "9999"; // merchant ID
string ECDSA_KEY = "2"; // private key ID used by bank to sign response

string URL = "https://moja.tatrabanka.sk/cgi-bin/e-commerce/start/example.jsp";
string RURL = "https://moja.tatrabanka.sk/cgi-bin/e-commerce/start/example.jsp";
string REM = ""; // e.g. "payment@example.com";

In [15]:
// signing keys helpers
#!share keys_json --from value 
var keys = JsonSerializer.Deserialize<JsonNode>(keys_json);
// keys.Display();

Func<string> KEY = () => keys["merchants"][MID]["KEY"].GetValue<string>();
Func<byte[]> KEYo = () => Utils.ConvertHexStringToByteArray(KEY());
Func<string> PRIVATE_PEM = () => keys["bank-keys"][ECDSA_KEY]["PRIVATE_PEM"]?.GetValue<string>();
Func<object> PRIVATE_PEMo = () => new PemReader(new StringReader(PRIVATE_PEM())).ReadObject();
Func<string> PUBLIC_PEM = () => keys["bank-keys"][ECDSA_KEY]["PUBLIC_PEM"].GetValue<string>();
Func<object> PUBLIC_PEMo = () => new PemReader(new StringReader(PUBLIC_PEM())).ReadObject();

## Request

```
Request = MID + AMT + CURR + VS + E2E + TXN + RURL + IPC + NAME + REM + TPAY + CID + ECID + TDS_* + TIMESTAMP + HMAC + AREDIR + LANG
P = MID + AMT + CURR + VS + E2E + TXN + RURL + IPC + NAME + REM + TPAY + CID + ECID + TDS_* + TIMESTAMP
HMAC = HMAC_SHA256(P)
```

In [16]:
// request parameters (P)
string TIMESTAMP = "01092014125505"; // DDMMYYYYhhmmss
string AMT = "1234.50"; // decimal dot
string CURR = "978"; // EUR
string VS = "1111";
string E2E = false ? "Y" : "N";
string NAME = "Jan Pokusny";
string IPC = "1.2.3.4";
string TXN = "";
string TPAY = "N";
string CID = "";
string ECID = "";

In [17]:
// HMAC
string HMAC_input = MID + AMT + CURR + VS + (E2E == "Y" ? "Y" : "") + TXN + RURL + IPC + NAME + REM + (TPAY == "Y" ? "Y" : "") + CID + ECID + TIMESTAMP;
display($"P = {HMAC_input}");

string HMAC_outputString = Utils.ConvertByteArrayToHexString( Utils.CalculateHMAC(HMAC_input, KEYo()) );
display($"HMAC = {HMAC_outputString}");

P = 99991234.509781111https://moja.tatrabanka.sk/cgi-bin/e-commerce/start/example.jsp1.2.3.4Jan Pokusny01092014125505

HMAC = 574b763f4afd4167b10143d71dc2054615c3fa76877dc08a7cc9592a741b3eb5

## Response

```
Response = AMT + CURR + VS + RES + AC + TID + TIMESTAMP + HMAC + ECDSA_KEY + ECDSA;
R = AMT + CURR + VS + RES + AC + TID + TIMESTAMP;
HMAC = HMAC_SHA256(R);
ECDSA = SHA256withECDSA(R + HMAC);
```

In [18]:
// request parameters (R)

// AMT, CURR, VS, TIMESTAMP must match exactly
string RES = "OK";
string AC = "123456";
string TID = "1";
string ECDSA_expected = "304502207a4e403a83a15dab431eb7e9c7862916d436948b182a8bcbedd9e0c8c60c07dd022100d693e438695aac16ab6c4e184d02c1b33856e63d0f965aeee4b11832ce8c1b04";

In [19]:
// HMAC
string HMAC_input2 = AMT + CURR + VS + RES + AC + TID + TIMESTAMP;
display($"R = {HMAC_input2}");
byte[] HMAC_output2 = Utils.CalculateHMAC(HMAC_input2, KEYo());
string HMAC_output2String = Utils.ConvertByteArrayToHexString(HMAC_output2);
display($"HMAC = {HMAC_output2String}");

string responseWithHmac = HMAC_input2 + HMAC_output2String;

R = 1234.509781111OK123456101092014125505

HMAC = 8df96c2603831046d0e3502cab1ddb7d9b629d7f09a44aee7abbec0be3f2d971

### Sign response

In [20]:
// sign with private key & verify signature with public key
string signatureString = null;

if (PRIVATE_PEM != null)
{
    byte[] signature = Utils.CalculateSignature(responseWithHmac, PRIVATE_PEM());
    signatureString = Utils.ConvertByteArrayToHexString(signature);
    display($"Signature = {signatureString}");
}
else
{
    display("No private key, using ECDSA_expected");
}

// verify signature with public key
bool isSignatureValid = Utils.IsSignatureValid(responseWithHmac, Utils.ConvertHexStringToByteArray(signatureString ?? ECDSA_expected), PUBLIC_PEM());
display($"Signature valid = {isSignatureValid}");

Signature = 3044022034e9f370bbe143e0cb427dc8cfc4aff683d6c72b3ee5cbee435c3c2193b6e78c0220372be5ced20bdae45fbc4c1bf8ff972e170b0cff2a1913d85ca5e0dabba5fb96

Signature valid = True